In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

In [2]:
# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [3]:
# Create aiport dataframe
cities_clean_df = pd.read_csv('Cities_Output2.csv')
cities_clean_df.dropna(inplace = True) 
cities_clean_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloud Coverage,Wind Speed,Country,Date
0,0,bethel,41.37,-73.41,64.99,97.0,100.0,4.94,US,1593842242
1,1,ust-nera,64.57,143.20,58.24,36.0,98.0,7.56,RU,1593842242
2,2,avarua,-21.21,-159.78,73.40,94.0,100.0,3.36,CK,1593842243
3,3,rikitea,-23.12,-134.97,72.00,70.0,12.0,16.04,PF,1593842244
4,6,bluff,-46.60,168.33,47.53,86.0,100.0,7.87,NZ,1593842245
...,...,...,...,...,...,...,...,...,...,...
551,604,nikolo-pavlovskoye,57.78,60.06,71.40,55.0,1.0,4.16,RU,1593842613
552,605,tessalit,20.20,1.01,86.81,12.0,32.0,9.91,ML,1593842524
553,606,caraballeda,10.61,-66.85,71.01,83.0,22.0,9.08,VE,1593842614
554,607,tyukhtet,56.54,89.30,72.95,85.0,100.0,6.85,RU,1593842615


In [4]:
locations = cities_clean_df[["Lat", "Lng"]]
humidity = cities_clean_df["Humidity"].astype(float)


In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
ideal_weather =cities_clean_df.loc[(cities_clean_df["Max Temp"] > 70) & (
    cities_clean_df["Max Temp"] < 80) & (
    cities_clean_df["Wind Speed"] < 10) & (
    cities_clean_df["Cloud Coverage"] ==0), :]

In [7]:
ideal_weather=pd.DataFrame(ideal_weather)
hotel=ideal_weather[["City", "Lat", "Lng", "Country"]]
hotel_df=pd.DataFrame(hotel)

#save cities only, build a new database


hotel_df["Hotel Name"]=''

#making sure maximum 10 rows
hotel_df=hotel_df.head(10)

hotel_df

,City,Lat,Lng,Country,Hotel Name
24,sovetskiy,61.36,63.58,RU,
167,archidona,37.10,-4.39,ES,
212,bud,39.45,-86.18,US,
221,bardiyah,31.76,25.09,LY,
240,konya,38.17,32.50,TR,
256,sinop,41.77,34.87,TR,
325,innisfail,-17.53,146.03,AU,
329,lusambo,-4.97,23.45,CD,
353,conde,-7.26,-34.91,BR,
413,kindu,-2.95,25.95,CD,


In [8]:
#Need the google geocode coordinates in order to ge the APis
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel.iterrows():
    params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
    "keyword":"hotel"
} 
    
    
    
    lat=str(row["Lat"])
    lng=str(row["Lng"])
    
    
    combined= lat + ',' ' ' + lng
    
    
    params['location'] = combined

    
    response = requests.get(base_url, params=params).json()
       
    try:
        hotel_df.loc[index, "Hotel Name"] =(response['results'][0]['name'])

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] =("No hotel within 5000m of coordinates")
   
            

    
   

In [9]:
hotel_df.dropna(inplace = True) 


In [10]:

 # Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [11]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [12]:
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…